In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, ADAM
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.utils import algorithm_globals
from qiskit.quantum_info import SparsePauliOp

from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN

algorithm_globals.random_seed = 42

# Training a Variational Quantum Classifier

We start with fixing some constants to create our data and circuit and consider `2` features and `2` qubits to solve a simple problem.

In [ ]:
feature_dimension = 2
qubit_count = 2

## Data Generation

We prepare a very simple dataset for our first classification task.

In [ ]:
num_samples = 20

X = 2 * algorithm_globals.random.random([num_samples, feature_dimension]) - 1
y01 = 1 * (np.sum(X, axis=1) >= 0)  # in { 0,  1}
y = 2 * y01 - 1  # in {-1, +1}
y_one_hot = np.zeros((num_samples, 2))
for i in range(num_samples):
    y_one_hot[i, y01[i]] = 1

for x, y_target in zip(X, y):
    if y_target == 1:
        plt.plot(x[0], x[1], "bo")
    else:
        plt.plot(x[0], x[1], "go")
plt.plot([-1, 1], [1, -1], "--", color="black")
plt.show()

## Classification

In the following, we create a circuit that consists from a data encoding part as well as a variational part. The data encoding part creates a feature map.

For our data encoding part we first want to consider *angle encoding*. So please take care to complete the following function. Ensure that all features are encoded. Note that the circuit might contain more qubits than features.

In [ ]:
def create_feature_map(feature_dimension=2, qubits=2):
    assert feature_dimension <= qubits
    data_parameter = ParameterVector("x", length=feature_dimension)

    feature_map = QuantumCircuit(qubits)
    # insert code for angle encoding here
    return feature_map

In [ ]:
feature_map = create_feature_map(feature_dimension, qubit_count)
ansatz = RealAmplitudes(qubit_count, reps=2).decompose(reps=2)

In [ ]:
initial_params = params = algorithm_globals.random.random(ansatz.num_parameters)
print(f"Starting with initial params {initial_params}")

We now compose our circuit from our two parts, the `feature_map` and our selected `ansatz`.

In [ ]:
# construct QNN
qc = QuantumCircuit(qubit_count)
qc.compose(feature_map, inplace=True)
qc.barrier()
qc.compose(ansatz, inplace=True)
qc.draw(output="mpl")

We now create a [quantum neural network](https://qiskit.org/documentation/machine-learning/stubs/qiskit_machine_learning.neural_networks.EstimatorQNN.html) based on the [`Estimator`](https://qiskit.org/documentation/stubs/qiskit.primitives.Estimator.html#qiskit.primitives.Estimator) primitive. Please note that this is doing exactly the same as we showed during our lecture. This is nothing else than our *variational classifier*.

In [ ]:
estimator_qnn = EstimatorQNN(
    circuit=qc, 
    input_params=feature_map.parameters, 
    weight_params=ansatz.parameters,
    observables=SparsePauliOp("ZI")
)

Trying out the forward step of our qnn...

In [ ]:
# QNN maps inputs to [-1, +1]
estimator_qnn.forward(X[0, :], algorithm_globals.random.random(estimator_qnn.num_weights))

... and our backward step.

In [ ]:
estimator_qnn.backward(X[0, :], params)

We now create a callback function. This is called for each iteration of the optimizer and will be passed two parameters: the current weights and the value of the objective function at those weights. We append the value of the objective function to an array so we can plot iteration versus objective function value and update the graph with each iteration.

In [ ]:
# callback function that draws a live plot when the .fit() method is called
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [ ]:
# construct neural network classifier
estimator_classifier = NeuralNetworkClassifier(
    estimator_qnn, 
    optimizer=COBYLA(maxiter=60), 
    callback=callback_graph, 
    loss="squared_error",
    initial_point=initial_params
)

In [ ]:
# create empty array for callback to store evaluations of the objective function
objective_func_vals = []
plt.rcParams["figure.figsize"] = (10, 4)

# fit classifier to data
estimator_classifier.fit(X, y)

# return to default figsize
plt.rcParams["figure.figsize"] = (6, 4)

# score classifier
estimator_classifier.score(X, y)

The following is a helper function to mark the data that has not been classified correctly.

In [ ]:
def evaluate_estimator(estimator, data, labels):
    # evaluate data points
    y_predict = estimator.predict(data)

    # plot results
    # red == wrongly classified
    for x, y_target, y_p in zip(data, labels, y_predict):
        try:
            y_target[1]
            current = y_target[1]
        except IndexError:
            current = y_target
        if current == 1:
            plt.plot(x[0], x[1], "bo")
        else:
            plt.plot(x[0], x[1], "go")
        if not np.all(y_target == y_p):
            plt.scatter(x[0], x[1], s=200, facecolors="none", edgecolors="r", linewidths=2)
    plt.plot([-1, 1], [1, -1], "--", color="black")
    plt.show()

In [ ]:
# evaluate data points
evaluate_estimator(estimator_classifier, X, y)

In [ ]:
estimator_classifier.weights

# Explicit implementation of Qiskits Variational Quantum Classifier

Qiskit also offers a specific variant for *Variational Quantum Classifiers*. Although the abbreviation *VQC* is commonly used for *variational quantum circuit*, this is not ment here. This function nearly does the same as the method presented above but considers one hot encoded labels that are used during training.

In [ ]:
# construct variational quantum classifier
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    loss="squared_error",  # "cross_entropy",
    optimizer=COBYLA(maxiter=60),
    callback=callback_graph, 
    initial_point=initial_params
)

In [ ]:
# create empty array for callback to store evaluations of the objective function
objective_func_vals = []
plt.rcParams["figure.figsize"] = (10, 4)

# fit classifier to data
vqc.fit(X, y_one_hot)

# return to default figsize
plt.rcParams["figure.figsize"] = (6, 4)

# score classifier
vqc.score(X, y_one_hot)

In [ ]:
evaluate_estimator(vqc, X, y_one_hot)

# Exercises

## Ansätze

Try out different ansätze to check how well you are training.

## Barren Plateaus

Adapt the number of qubits for the given circuit and evaluate if and when you are facing a *barren plateau*.

Take care that you also need to adapt the `observable` by attaching *Identity* as often as the number of qubits - 1 in case you still execute your classifier with a Z measurement.

# Optional Homework Exercise

Use the [`ad_hoc_data`](https://qiskit.org/documentation/machine-learning/stubs/qiskit_machine_learning.datasets.ad_hoc_data.html#qiskit_machine_learning.datasets.ad_hoc_data) method from `qiskit_machine_learning` and generate a toy dataset according to the procedure outlined in the paper by [Havlicek et al.](https://arxiv.org/pdf/1804.11326.pdf). This dataset can theoretically be fully separated by using the [`ZZFeatureMap`](https://qiskit.org/documentation/stable/0.24/stubs/qiskit.circuit.library.ZZFeatureMap.html?highlight=zzfeaturemap#qiskit.circuit.library.ZZFeatureMap).

Build a circuit and implement a training to see which accuracy you are able to reach for the test dataset.